In [ ]:
import json
import os
from openai import OpenAI
from tqdm import tqdm

In [ ]:
import sys
sys.path.append('/Users/mn/Desktop/BridgeAthletics/Proj1/DataPreProcessing')
from Functions.DataSetManipulation import json_read


In [ ]:
##################################################################

In [ ]:
with open("/Users/mn/Desktop/BridgeAthletics/config.json", "r") as config_file:
    config = json.load(config_file)
    api_key = config["OPENAI_API_KEY"]

client = OpenAI(api_key=api_key)

In [ ]:
def run_chatgpt(prompt, client, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    return response.choices[0].message.content

In [ ]:
############################ CHOOSE APPROPRIATE DATA SET DIRECTORY #############################

In [ ]:
dir='/Users/mn/Desktop/BridgeAthletics/Dataset2_allparams/'

In [ ]:
############################ REMOVING USELESS BLOCK NAMES ####################################

In [ ]:
json_file = dir+'initialdataset.json'
with open(json_file, "r") as file:
    json_data = json.load(file)

In [ ]:
print(len(json_data))

In [ ]:
for entry in json_data[:5]:
    block_name = entry["input"]
    prompt = f"Does the workout title '{block_name}' clearly describe the focus or purpose of the workout? Reply only with 'yes' or 'no'."
    print(prompt)
    print("\nOutput:")
    print(">>", run_chatgpt(prompt, client))
    print("\n-------------------------")

In [ ]:
for i, entry in tqdm(enumerate(json_data), total=len(json_data)):
    block_name = entry["input"]
    prompt = f"Does the workout title '{block_name}' clearly describe the focus or purpose of the workout? Reply only with 'yes' or 'no'."
    json_data[i]["useful"] = run_chatgpt(prompt, client)

In [ ]:
n=0
for i in range(0,len(json_data)):
    if (json_data[i]["useful"]!="Yes") and (json_data[i]["useful"]!="No"):
        print(json_data[i]["useful"])
    if json_data[i]["useful"]=="No":
        n=n+1

print("no:",n,"yes:",len(json_data)-n)

In [ ]:
#############RUN ONCE##########################

In [ ]:
new_json_file = dir+"dataset-usefulYesNo.json"
print(new_json_file)
if os.path.exists(new_json_file):
    m=input("are you sure you want to overwrite file? reply with 'yes' or 'no'")
    if m.lower()=='yes':
        with open(new_json_file, "w") as file:
            json.dump(json_data, file, indent=1)  
else:
    with open(new_json_file, "w") as file:
        json.dump(json_data, file, indent=1)  

In [ ]:
#################################################

In [ ]:
json_file = dir+"dataset-usefulYesNo.json"
with open(json_file, "r") as file:
    json_data = json.load(file)
print(len(json_data))

In [ ]:
filtered_data = [item for item in json_data if item["useful"] == "Yes"]
print(len(filtered_data))

In [ ]:
#############RUN ONCE##########################

In [ ]:
new_json_file = json_file.replace("-usefulYesNo.json", "-Yes.json")
with open(new_json_file, "w") as file:
    json.dump(filtered_data, file, indent=1) 

In [ ]:
#######################################

In [ ]:
####################### EXTRACTING KEY INFO FROM BLOCK NAMES #################################

In [ ]:
json_file = dir+"dataset-Yes.json"
with open(json_file, "r") as file:
    json_data = json.load(file)
print(len(json_data))

In [ ]:
print(json_data[0])

In [ ]:
for entry in json_data[:5]:
    block_name = entry["input"]
    prompt = f"For the workout title: '{block_name}', extract only the key phrases that describe the focus or type of the workout, expanding any abbreviations and removing any numerical details and session identifiers like 'day' or 'week' or 'min'. Reply only with the title."
    print(prompt)
    print("\nOutput:")
    print(">>", run_chatgpt(prompt, client))
    print("\n-------------------------")

In [ ]:
for i, entry in tqdm(enumerate(json_data), total=len(json_data)):
    block_name = entry["input"]
    prompt = f"For the workout title: '{block_name}', extract only the key words that describe the focus or type of the workout, expanding any abbreviations and removing any numerical details and session identifiers like dates, 'level', 'phase', 'day' or 'week'. Reply only with the title."
    json_data[i]["new_input"] = run_chatgpt(prompt, client)

In [ ]:
json_file = dir+"dataset_gpt_prefinal.json"
if os.path.exists(json_file):
    m=input("are you sure you want to overwrite file? reply with 'yes' or 'no'")
    if m.lower()=='yes':
        with open(json_file, "w") as file:
            json.dump(json_data, file, indent=1)
else:
    with open(json_file, "w") as file:
        json.dump(json_data, file, indent=1)

In [ ]:
###################### DATA CLEANING: REMOVING REMAINING BAD ENTRIES ##########################

In [ ]:
import re
def contains_keywords_or_number(s):
    keywords = ["title", "week", "day", "phase","min","dirty"]
    # Check for keywords
    if any(keyword in s.lower() for keyword in keywords):
        return True
    # Check for numbers
    if re.search(r'\d', s):
        return True
    if s.lower()=="block":
        return True
    return False

In [ ]:
with open(json_file, "r") as file:
    json_data = json.load(file)
print(len(json_data),json_data[0])

In [ ]:
inc_arr=[]
for i in range(len(json_data)):
    if contains_keywords_or_number(json_data[i]["new_input"]):
        print (i,json_data[i]["new_input"])
        inc_arr.append(i)
print([(i,j) for i,j in enumerate((inc_arr))])

In [ ]:
for i, entry in tqdm(enumerate(inc_arr),total=len(inc_arr)):
    block_name = json_data[entry]["new_input"]
    prompt = f"For the workout title: '{block_name}', extract only the key words that describe the focus or type of the workout, session information like dates or 'level' or 'minutes' or 'phase'. Reply only with the title."
    json_data[entry]["new_input"] = run_chatgpt(prompt, client)

In [ ]:
inc_arr=[]
for i in range(len(json_data)):
    if contains_keywords_or_number(json_data[i]["new_input"]):
        print (i,json_data[i]["new_input"])
        inc_arr.append(i)
print([(i,j) for i,j in enumerate((inc_arr))])

In [ ]:
to_remove=[2319,2177,2146,1504,1469,444,510,588,619]

In [ ]:
#ONLY RUN ONCE
for i in sorted(inc_arr, reverse=True):
    if i in to_remove:
        del json_data[i]
# print(json_data[1500])
# print(len(json_data))

In [ ]:
print(len(json_data))

In [ ]:
for item in json_data:
    item['input'] = item['new_input']
    del item['new_input']
    del item['useful']

In [ ]:
print(json_data[0])

In [ ]:
#RUN CELL TWICE
filtered_json_data = []

for i in range(len(json_data)):
    # Find isolated 'A', 'B', or 'C' in the 'input' field
    matches = re.findall(r'\b[A-C]\b', json_data[i]['input'])
    if matches:
        if json_data[i]['input'][-2:] in [' A',' B',' C', ' D', ' E']:
            json_data[i]['input']=json_data[i]['input'][:len(json_data[i]['input'])-2]
        print(json_data[i]['input'])
    if not matches:
        filtered_json_data.append(json_data[i])

In [ ]:
f_json_data = filtered_json_data
del filtered_json_data

In [ ]:
print(len(f_json_data))

In [ ]:
for i in range(len(f_json_data) - 1, -1, -1):
    if f_json_data[i]['input'].lower().startswith('block') or f_json_data[i]['input'].lower().startswith('test'):
        del f_json_data[i]

In [ ]:
print(len(f_json_data))

In [ ]:
# for i in f_json_data:
#     print(i['input'])

In [ ]:
json_file = dir+"datasetprefinal.json"
if os.path.exists(json_file):
    m=input("are you sure you want to overwrite file? reply with 'yes' or 'no'")
    if m.lower()=='yes':
        with open(json_file, "w") as file:
            json.dump(f_json_data, file, indent=1)
else:
    with open(json_file, "w") as file:
        json.dump(f_json_data, file, indent=1)

In [ ]:
############################### REMOVING DUPLICATES ########################################

In [ ]:
import argparse
from sklearn import __version__ as sklearn_version
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
json_file = dir+"datasetprefinal.json"
with open(json_file, "r") as file:
    f_json_data = json.load(file)
print(len(f_json_data))

In [ ]:
def remove_caps_punct(input):
    input = re.sub(r'[^\w\s]', '', input)
    input = input.lower()
    return input

In [ ]:
def two_strings_cosine_similarity(string1, string2):
    vectorizer = TfidfVectorizer(stop_words=None, analyzer='char', ngram_range=(1, 3))
    tfidf_matrix = vectorizer.fit_transform([remove_caps_punct(string1).lower(), remove_caps_punct(string2).lower()])
    cos_sim_matrix = cosine_similarity(tfidf_matrix)
    return cos_sim_matrix[0, 1]

In [ ]:
def find_group_duplicates(json_data, threshold=0.9, key="input"):
    if key!="output":
        items = [remove_caps_punct(item[key]) for item in json_data]
    else:
        items = [remove_caps_punct(str(item[key])) for item in json_data]
        
    vectorizer = TfidfVectorizer(stop_words=None, analyzer='char', ngram_range=(1, 3))
    tfidf = vectorizer.fit_transform(items)

    cos_sim_matrix = cosine_similarity(tfidf)

    dup_groups = {}
    unique_groups = {}
    visited = set()

    for i in range(len(cos_sim_matrix)):
        if i in visited:
            continue
            
        group = [i]
        for j in range(i + 1, len(cos_sim_matrix)):
            if cos_sim_matrix[i, j] > threshold:
                group.append(j)
                visited.add(j)
        
        if len(group) > 1:
            dup_groups[i] = group
            visited.add(i)

        else:
            unique_groups[i] = group
            
            
    duplicate_groups = [[json_data[id] for id in group] for group in dup_groups.values()]
    unique_groups = [[json_data[id] for id in group] for group in unique_groups.values()]

    return duplicate_groups, unique_groups


In [ ]:
def find_out_duplicates(json_data, key="output"):
    unique_out = []
    unique_obj = []
    output_dict = {}

    for item in json_data:
        output = item[key]
        output_str=json.dumps(item[key])
        input_block = item["input"]

        if output in unique_out:
            similar = False
            for existing_input in output_dict[output_str]:
                if two_strings_cosine_similarity(input_block, existing_input) >= 0.9:
                    similar = True
                    break
            if not similar:
                unique_obj.append(item)
                output_dict[output_str].append(input_block)
        else:
            unique_out.append(output)
            unique_obj.append(item)
            output_dict[output_str] = [input_block]

    return unique_out, unique_obj


In [ ]:
duplicate_groups, unique_groups = find_group_duplicates(f_json_data, threshold=1, key="output")
t=0
if duplicate_groups:
    print("Duplicate Groups Found:")
    for group_id, group in enumerate(duplicate_groups, 1):
        # print(f"Group {group_id}:")
        # print([group[i]['input'] for i in range (0,len(group))])
        t=t+len(group)
        g=group_id
print("number of samples in duplicates:",t,"number of groups",g)

t=0
if unique_groups:
    print("Unique Groups Found:")
    for group_id, group in enumerate(unique_groups, 1):
        # print(f"Group {group_id}:")
        # print(group[0]['input'])
        # print(len(group))
        t=t+len(group)
        g=group_id
print("number of samples in uniques:",t,"number of groups",g)
        


In [ ]:
unique_outs,removed_dupli=find_out_duplicates(f_json_data)
print("unique outputs/blocks:",len(unique_outs),"number of duplicate pairs:",len(f_json_data)-len(removed_dupli))

In [ ]:
f_json_data = removed_dupli

In [ ]:
new_json_file = dir+"finaldataset.json"
with open(new_json_file, "w") as file:
    json.dump(f_json_data, file, indent=1) 

In [ ]:
json_file = dir+"finaldataset.json"
with open(json_file, "r") as file:
    final_json_data = json.load(file)
print(len(final_json_data))

In [ ]:
# OPTIONAL Removing blocks that are too long
count=0
for i in range(len(final_json_data) - 1, -1, -1):
    if len(final_json_data[i]["output"]) >= 6:
        del final_json_data[i]
        count+=1
print(count)

In [ ]:
new_json_file = dir+"finaldataset_shortblocks.json"
with open(new_json_file, "w") as file:
    json.dump(final_json_data, file, indent=1)

In [ ]:
json_file = dir+"finaldataset_shortblocks.json"
with open(json_file, "r") as file:
    final_json_data = json.load(file)
print(len(final_json_data))

In [ ]:
json_file = dir+"finaldataset_shortblocks.json"
with open(json_file, "r") as file:
    final_json_data = json.load(file)
print(len(final_json_data))

In [ ]:
############################## QUICK DATA ANALYSIS ##############################

In [ ]:
def check_parameters_correctness(model_outputs_list, param_list):
    required_keys = {'exercise', 'sets'}
    c=0
    for i, outer_list in enumerate(model_outputs_list):
        for j, dictionary in enumerate(outer_list):
            if not required_keys.issubset(dictionary.keys()):
                print(f"ERROR: Dictionary at index [{i}][{j}] must have at least the keys {required_keys}.")
                c=c+1
                continue
            
            if not any(param in dictionary for param in param_list):
                print(f"ERROR: Dictionary at index [{i}][{j}] must have at least one of the keys from {param_list}.")
                c=c+1
                continue
            
            allowed_keys = required_keys.union(param_list)
            if not set(dictionary.keys()).issubset(allowed_keys):
                print(f"ERROR: Dictionary at index [{i}][{j}] must only have keys from {allowed_keys}.")
                c=c+1
                continue
    print(f"inconsistency in keys for {c} out of {len(model_outputs_list)} ")

In [ ]:
check_parameters_correctness(data_outputs,param_list)

In [ ]:
def count_param_frequency(model_outputs_list, param_list):
    param_counts = {param: 0 for param in param_list}
    
    for outer_list in model_outputs_list:
        for param in param_list:
            if any(param in dictionary for dictionary in outer_list):
                param_counts[param] += 1
    
    total_outer_lists = len(model_outputs_list)
    param_percentages = {param: (count / total_outer_lists) * 100 for param, count in param_counts.items()}
    
    return param_counts, param_percentages


param_counts, param_percentages = count_param_frequency(data_outputs, param_list)
print("Parameter Counts:", param_counts)
print("Parameter Percentages:", param_percentages)

In [ ]:
import ast
import pandas as pd
import numpy as np

In [ ]:
#TO EDIT AS YOU INCREASE PARAMS
param_list=['reps','time','distance']

In [ ]:
def output_list_to_df(model_outputs_list,param_list):
    flat_data = []
    for i, outer_list in enumerate(model_outputs_list):
        for d in outer_list:
            row = [i, d['exercise'], d['sets']]
            for param in param_list:
                row.append(d.get(param, None))
            flat_data.append(row)
    columns = ['block', 'exercise', 'sets'] + param_list
    
    df = pd.DataFrame(flat_data, columns=columns)
    return df

In [ ]:
data_outputs = [final_json_data[i]['output'] for i in range(len(final_json_data))]
main_dataset_df = output_list_to_df(data_outputs,param_list)
main_dataset_df

In [ ]:
total_number_of_ex = main_dataset_df['exercise'].count()
max_number_of_ex_per_block = main_dataset_df.groupby('block')['exercise'].count().max()
average_number_of_ex_per_outer_list = main_dataset_df.groupby('block')['exercise'].count().mean()
median_number_of_ex_per_outer_list = main_dataset_df.groupby('block')['exercise'].count().median()
std_dev_of_ex_per_outer_list = main_dataset_df.groupby('block')['exercise'].count().std()
number_of_blocks_with_one_exercise = (main_dataset_df.groupby('block')['exercise'].count() == 1).sum()


print(f"Total number of exercises: {total_number_of_ex}")
print(f"Max number of exercises per block: {max_number_of_ex_per_block}")
print(f"Average number of exercises per block: {average_number_of_ex_per_outer_list}")
print(f"Median number of exercises per block: {median_number_of_ex_per_outer_list}")
print(f"Standard deviation of exercises per block: {std_dev_of_ex_per_outer_list}")
print(f"Percentage of blocks with only one exercise: {number_of_blocks_with_one_exercise/len(final_json_data) *100}%")

In [ ]:
total_sets = main_dataset_df['sets'].sum()
max_sets_per_block = main_dataset_df.groupby('block')['sets'].sum().max()
average_sets_per_outer_list = main_dataset_df.groupby('block')['sets'].sum().mean()
median_sets_per_outer_list = main_dataset_df.groupby('block')['sets'].sum().median()
std_dev_sets_per_outer_list = main_dataset_df.groupby('block')['sets'].sum().std()

print(f"Total number of sets: {total_sets}")
print(f"Max number of sets per block: {max_sets_per_block}")
print(f"Average number of sets per block: {average_sets_per_outer_list}")
print(f"Median number of sets per block: {median_sets_per_outer_list}")
print(f"Standard deviation of sets per block: {std_dev_sets_per_outer_list}")